## Simple Mission in Auto-Mode

This notebook serves to check the ardupilot installation

In [1]:
import os

from config import (
    ARDU_LOGS_PATH,
    ARDUPILOT_VEHICLE_PATH,
    ENV_CMD_PYT,
    VEH_PARAMS_PATH,
    BasePort,
)
from helpers import clean, create_process, setup_logging, wait_for_port
from helpers.connections.mavlink.conn import create_tcp_conn
from helpers.coordinates import GRAPose
from plan import State
from plan.planner import Plan

clean()

## Launch Copter (ardupilot)

In [2]:
#This must agree with first waypoint in mission.waypoint
gra_origin = GRAPose(lat=-35.3633245,lon=149.1652241,alt=1/3_000_234,heading=0)
spawn_str= gra_origin.to_str()



sim_vehicle_path = os.path.expanduser(ARDUPILOT_VEHICLE_PATH)
sysid = 1
vehicle_cmd = (
    f"python3 {sim_vehicle_path} "
    f"-v ArduCopter "
    f"-I{sysid-1} --sysid {sysid} "
    f"--no-rebuild "
    f"--use-dir={ARDU_LOGS_PATH} "
    f"--add-param-file {VEH_PARAMS_PATH} "
    f"--no-mavproxy "
    f"--port-offset=0 "  # ArduPilot automatically binds TCP port 5760 (BasePort.ARP)
    f'--custom-location={spawn_str}'
)

create_process(
                vehicle_cmd,
                after="exec bash",
                visible= True,
                suppress_output=False,
                title="ardu_vehicle",
                env_cmd=ENV_CMD_PYT,
            )
wait_for_port(port = BasePort.ARP, timeout = 0.5)



Waiting for port 5760 to open...


## 1. Connect to the vehicle

In [3]:
conn = create_tcp_conn(
    base_port=BasePort.ARP, 
    offset=0,
    role="client", 
    sysid=sysid
)

print("✅ TCP connection established!")

✅ TCP connection established!


## Create auto plan from mission file

The mission is loaded from "plan/missions/simple_mission.waypoints"

In [4]:
plan = Plan.auto(name='auto',
                mission_path="plan/missions/simple_mission.waypoints")
plan

🕓 <Plan '📋 auto'>
  🕓 <Action '💾 UPLOAD_MISSION'>
    🕓 <ClearMission '🔹 clear previous mission'>
    🕓 <UploadMission '🔹 upload mission'>
  🕓 <Action '🛡️ PREARM'>
    🕓 <CheckDisarmed '🔹 Check disarmed'>
    🕓 <EKFStatus '🔹 Check EKF status'>
    🕓 <GPSStatus '🔹 Check GPS'>
    🕓 <CheckSystem '🔹 Check system status'>
  🕓 <Action '⚙️ MODE'>
    🕓 <SwitchMode '🔹 Switch to GUIDED'>
  🕓 <Action '🔒 ARM'>
    🕓 <Arm '🔹 arm'>
  🕓 <Action '🚀 START_MISSION'>
    🕓 <StartMission '🔹 start mission'>
  🕓 <Action '👀 MONITOR_MISSION'>
    🕓 <CheckItems '🔹 check items'>
    🕓 <CheckEndMission '🔹 check end mission'>

# Execute Auto Plan

In [5]:
plan.bind(conn,gra_origin.unpose())
setup_logging('plan',verbose=1)
while plan.state != State.DONE:
    plan.act()

15:25:23 - plan - INFO - 🧹 Vehicle 1: Cleared previous mission
15:25:23 - plan - INFO - ✅ Vehicle 1: 4 waypoints read
15:25:25 - plan - INFO - ✅ Vehicle 1: Mission upload successful!
15:25:25 - plan - INFO - ✅ Vehicle 1: Action Done: 💾 UPLOAD_MISSION
15:26:06 - plan - INFO - ✅ Vehicle 1: Action Done: 🛡️ PREARM
15:26:06 - plan - INFO - ✅ Vehicle 1: Action Done: ⚙️ MODE
15:26:07 - plan - INFO - ✅ Vehicle 1: Action Done: 🔒 ARM
15:26:07 - plan - INFO - 🚀 Vehicle 1: Mission has started
15:26:07 - plan - INFO - ✅ Vehicle 1: Action Done: 🚀 START_MISSION
15:26:07 - plan - INFO - 📦 Vehicle 1 has 4 mission items
15:26:08 - plan - INFO - Vehicle 1: ⭐ Reached item: 0
15:26:08 - plan - INFO - Vehicle 1: 📍 Target Position: ENU(x=-0.18, y=-0.28, z=5.0)
15:26:14 - plan - INFO - Vehicle 1: ⭐ Reached item: 1
15:26:14 - plan - INFO - Vehicle 1: 📍 Target Position: ENU(x=-0.18, y=-6.21, z=5.0)
15:26:21 - plan - INFO - Vehicle 1: ⭐ Reached item: 2
15:26:21 - plan - INFO - Vehicle 1: 📍 Target Position: ENU(x

In [6]:
plan

✅ <Plan '📋 auto'>
  ✅ <Action '💾 UPLOAD_MISSION'>
    ✅ <ClearMission '🔹 clear previous mission'>
    ✅ <UploadMission '🔹 upload mission'>
  ✅ <Action '🛡️ PREARM'>
    ✅ <CheckDisarmed '🔹 Check disarmed'>
    ✅ <EKFStatus '🔹 Check EKF status'>
    ✅ <GPSStatus '🔹 Check GPS'>
    ✅ <CheckSystem '🔹 Check system status'>
  ✅ <Action '⚙️ MODE'>
    ✅ <SwitchMode '🔹 Switch to GUIDED'>
  ✅ <Action '🔒 ARM'>
    ✅ <Arm '🔹 arm'>
  ✅ <Action '🚀 START_MISSION'>
    ✅ <StartMission '🔹 start mission'>
  ✅ <Action '👀 MONITOR_MISSION'>
    ✅ <CheckItems '🔹 check items'>
    ✅ <CheckEndMission '🔹 check end mission'>